In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-08"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7404,2024-10-08,Eua Nba,20:00,Charlotte Hornets,Miami Heat,1.73,2.00,218.5,1.85,1.93,-1.5,1.89,1.89,vBmkz9iE,0.578035,0.500000,0.540541,0.518135,0.078035,0.0,1.0,NaN,NaN,200.530,84.487417,0.421321,256.600,105.578164,0.411450,310.932,222.734,146.16,204.68,0.0,0.0,0.0,0.0,0.102369,0.029930,0.000000,6.02,1.204,0.606312,0.356397,0.4,0.043603,2.66,0.532,1.879699,0.440838,0.8,0.359162
7405,2024-10-08,Eua Nba,20:00,Cleveland Cavaliers,Chicago Bulls,1.55,2.36,216.5,1.88,1.90,-3.5,1.95,1.83,0QybYnMQ,0.645161,0.423729,0.531915,0.526316,0.068890,0.8,0.6,NaN,NaN,247.108,106.047831,0.429156,179.278,31.940232,0.178160,244.546,251.120,169.15,129.36,0.0,0.0,0.0,0.0,0.292970,0.007483,0.044896,-1.88,-0.376,-1.462766,0.536437,0.4,-0.136437,0.60,0.120,11.333333,0.467355,0.5,0.032645
7406,2024-10-08,Eua Nba,20:00,Detroit Pistons,Phoenix Suns,1.45,2.62,222.5,1.88,1.91,-4.5,1.90,1.88,t01N4NLM,0.689655,0.381679,0.531915,0.523560,0.071335,0.6,0.6,NaN,NaN,172.170,19.520666,0.113380,182.276,36.304667,0.199174,193.448,196.892,150.30,141.68,0.0,0.0,0.0,0.0,0.406543,0.011194,0.007483,1.10,0.220,2.045455,0.452837,0.5,0.047163,-1.20,-0.240,-6.750000,0.528220,0.5,-0.028220
7407,2024-10-08,Eua Nba,20:30,Atlanta Hawks,Indiana Pacers,2.09,1.69,227.5,1.93,1.85,1.5,1.89,1.89,UuSTSQql,0.478469,0.591716,0.518135,0.540541,0.070185,0.4,0.6,NaN,NaN,182.146,54.414563,0.298741,283.146,184.139886,0.650335,193.126,280.812,141.90,578.56,0.0,1.0,0.0,0.0,0.149652,0.029930,0.000000,-1.24,-0.248,-4.395161,0.554392,0.5,-0.054392,-2.98,-0.596,-1.157718,0.526841,0.3,-0.226841
7408,2024-10-08,Eua Nba,23:30,Los Angeles Clippers,Brooklyn Nets,1.41,2.75,217.5,1.93,1.85,-5.5,1.95,1.84,Q7LxR4E0,0.709220,0.363636,0.518135,0.540541,0.072856,0.4,0.8,NaN,NaN,228.570,88.387250,0.386697,165.440,29.821787,0.180257,196.302,253.128,160.08,151.32,0.0,0.0,0.0,0.0,0.455540,0.029930,0.041046,-1.33,-0.266,-1.541353,0.533068,0.3,-0.233068,-1.32,-0.264,-6.628788,0.448368,0.4,-0.048368
7409,2024-10-08,Europa Eurocopa,13:00,BC Wolves,Ulm,1.59,2.26,168.5,1.85,1.85,-5.5,2.02,1.61,raOdVdom,0.628931,0.442478,0.540541,0.540541,0.071409,0.6,0.6,NaN,NaN,312.834,247.710033,0.791826,168.178,56.209500,0.334226,171.634,159.808,128.34,138.24,0.0,0.0,0.0,0.0,0.246110,0.000000,0.159732,-2.34,-0.468,-1.260684,0.531184,0.4,-0.131184,1.24,0.248,5.080645,0.000000,0.0,0.000000
7410,2024-10-08,Europa Eurocopa,13:00,Lietkabelis,Aris,1.58,2.29,153.5,1.85,1.85,-5.5,2.02,1.61,fNX3xWbU,0.632911,0.436681,0.540541,0.540541,0.069593,1.0,0.8,NaN,NaN,219.246,70.819316,0.323013,192.062,68.526653,0.356794,206.178,301.116,187.22,282.75,0.0,0.0,0.0,0.0,0.259455,0.000000,0.159732,1.46,0.292,1.986301,0.000000,0.0,0.000000,0.92,0.184,7.010870,0.000000,0.0,0.000000
7411,2024-10-08,Europa Eurocopa,13:30,Turk Telekom,Cedevita Olimpija,1.50,2.47,158.5,1.85,1.85,-6.5,2.02,1.61,M9ZevhUH,0.666667,0.404858,0.540541,0.540541,0.071525,0.2,1.0,NaN,NaN,173.388,139.639970,0.805361,308.948,181.973059,0.589009,110.914,442.188,85.68,246.45,0.0,0.0,1.0,0.0,0.345538,0.000000,0.159732,-1.05,-0.210,-2.380952,0.000000,0.0,0.000000,-0.04,-0.008,-183.750000,0.000000,0.0,0.000000
7412,2024-10-08,Europa Eurocopa,15:00,Besiktas,Hapoel Tel-Aviv,2.45,1.51,168.5,1.85,1.85,2.5,2.02,1.61,vRH5TzGa,0.408163,0.662252,0.540541,0.540541,0.070415,0.6,0.2,NaN,NaN,136.554,21.238491,0.155532,155.400,63.571171,0.409081,125.880,156.492,134.42,111.00,0.0,0.0,0.0,0.0,0.335697,0.000000,0.159732,-2.38,-0.476,-3.04

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
